In [1]:
import json
import sys
import globus_sdk

jsonFile = "globus_task_list.json"

In [2]:
CLIENT_ID='2826c2c1-2bdf-4ea4-8f3a-d25487ad9c4c'
client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
client.oauth2_start_flow_native_app()

authorize_url = client.oauth2_get_authorize_url()
print('Please go to this URL and login: {0}'.format(authorize_url))

# this is to work on Python2 and Python3 -- you can just use raw_input() or
# input() for your specific version
get_input = getattr(__builtins__, 'raw_input', input)
auth_code = get_input(
    'Please enter the code you get after login here: ').strip()
token_response = client.oauth2_exchange_code_for_tokens(auth_code)

globus_auth_data = token_response.by_resource_server['auth.globus.org']
globus_transfer_data = token_response.by_resource_server['transfer.api.globus.org']

# most specifically, you want these tokens as strings
AUTH_TOKEN = globus_auth_data['access_token']
TRANSFER_TOKEN = globus_transfer_data['access_token']

Please go to this URL and login: https://auth.globus.org/v2/oauth2/authorize?client_id=2826c2c1-2bdf-4ea4-8f3a-d25487ad9c4c&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=openid+profile+email+urn%3Aglobus%3Aauth%3Ascope%3Atransfer.api.globus.org%3Aall&state=_default&response_type=code&code_challenge=RUWEmxDlSqiWEetErmWs85EvtUaM9pKarfqw3HeYkkg%3D&code_challenge_method=S256&access_type=online
Please enter the code you get after login here: PBtRaxeWivA1oucrlpvQVOTjgvt2Zg


In [3]:
# a GlobusAuthorizer is an auxiliary object we use to wrap the token. In
# more advanced scenarios, other types of GlobusAuthorizers give us
# expressive power
authorizer = globus_sdk.AccessTokenAuthorizer(TRANSFER_TOKEN)
tc = globus_sdk.TransferClient(authorizer=authorizer)

# high level interface; provides iterators for list responses
print("My Endpoints:")
for ep in tc.endpoint_search(filter_scope="my-endpoints"):
    print("[{}] {}".format(ep["id"], ep["display_name"]))

My Endpoints:
[5cd8da92-d68a-11e6-9ccd-22000a1e3b52] galen testshare on ncsa#Nearline
[d413b154-8db0-11e6-b054-22000b92c261] macbook at home
[cbfb1a23-6d04-11e5-ba46-22000b92c6ec] None
[d1763ade-6d04-11e5-ba46-22000b92c6ec] None
[cbfb19f5-6d04-11e5-ba46-22000b92c6ec] None
[c1cf11eb-6d04-11e5-ba46-22000b92c6ec] None
[9459c985-6d04-11e5-ba46-22000b92c6ec] None
[c00a495d-6d04-11e5-ba46-22000b92c6ec] None
[cabacc80-6d04-11e5-ba46-22000b92c6ec] None
[79f0f457-6d04-11e5-ba46-22000b92c6ec] None
[82d9c1ea-6d04-11e5-ba46-22000b92c6ec] None
[e26200a2-6d04-11e5-ba46-22000b92c6ec] None
[ce71c6f2-6d04-11e5-ba46-22000b92c6ec] None
[cbfb1a08-6d04-11e5-ba46-22000b92c6ec] None
[74b5c063-6d04-11e5-ba46-22000b92c6ec] None
[78c782b7-6d04-11e5-ba46-22000b92c6ec] None
[7bb92a0d-6d04-11e5-ba46-22000b92c6ec] None
[a4fcc57c-6d04-11e5-ba46-22000b92c6ec] None
[777b99be-6d04-11e5-ba46-22000b92c6ec] None
[db57dd3a-6d04-11e5-ba46-22000b92c6ec] None
[b9d021ce-6d04-11e5-ba46-22000b92c6ec] None
[cb63d7e8-6d04-11e5-ba4

In [2]:
f= open(jsonFile)
data= json.load(f)


In [4]:
print("My Last 25 Tasks:")
# `filter` to get Delete Tasks (default is just Transfer Tasks)
for task in tc.task_list(num_results=25, filter="type:TRANSFER,DELETE"):
    print(task["task_id"], task["type"], task["status"])


My Last 25 Tasks:
83f1c9fa-f9d4-11e6-ba9e-22000b9a448b TRANSFER SUCCEEDED
41978644-f9d4-11e6-ba9e-22000b9a448b TRANSFER SUCCEEDED
9f5467a2-e192-11e6-9d3a-22000a1e3b52 TRANSFER SUCCEEDED
de2554aa-e183-11e6-9d39-22000a1e3b52 TRANSFER SUCCEEDED
d60dd918-e183-11e6-9d39-22000a1e3b52 TRANSFER SUCCEEDED
d1462764-e183-11e6-9d39-22000a1e3b52 TRANSFER SUCCEEDED
9a5f8900-de56-11e6-9d15-22000a1e3b52 TRANSFER SUCCEEDED
f159b708-ddb8-11e6-9d11-22000a1e3b52 TRANSFER SUCCEEDED
ef7d4bb4-ddb8-11e6-9d11-22000a1e3b52 DELETE SUCCEEDED
cb677422-ddb6-11e6-9d11-22000a1e3b52 TRANSFER SUCCEEDED
95d64cee-d68a-11e6-9ccd-22000a1e3b52 DELETE SUCCEEDED
d40abdd4-d689-11e6-9ccd-22000a1e3b52 DELETE SUCCEEDED
be946b4e-ad08-11e6-9af4-22000a1e3b52 TRANSFER SUCCEEDED
f60ae604-a5db-11e6-9ace-22000a1e3b52 TRANSFER SUCCEEDED
bf355276-a5cd-11e6-9ace-22000a1e3b52 TRANSFER SUCCEEDED
083c3ee8-a5ca-11e6-9ace-22000a1e3b52 TRANSFER SUCCEEDED
89020e40-9b8e-11e6-b0c6-22000b92c261 TRANSFER SUCCEEDED
72246e74-9b8a-11e6-b0c6-22000b92c261

In [11]:
for transfer in data['DATA']:
#for transfer in tc.task_list(num_results=5):
    print("=================================================================")
    print("task_id= ( %s ) %s -> %s" % (transfer['task_id'],transfer['source_endpoint_display_name'],transfer['destination_endpoint_display_name']))
    print("request_time %s" % transfer['request_time'])
    print("completed    %s" % transfer['completion_time'])
    print("type=%s Mbytes=%.1f Mbytes/s=%.1f" % (transfer['type'],transfer['bytes_transferred']/1024,transfer['effective_bytes_per_second']/1024))

task_id= ( 83f1c9fa-f9d4-11e6-ba9e-22000b9a448b ) ncsa#BlueWaters -> ncsa#jyc
request_time 2017-02-23 14:29:43+00:00
completed    2017-02-23 14:29:50+00:00
type=TRANSFER Mbytes=12219.0 Mbytes/s=1624.0
task_id= ( 41978644-f9d4-11e6-ba9e-22000b9a448b ) ncsa#BlueWaters -> ncsa#jyc
request_time 2017-02-23 14:27:51+00:00
completed    2017-02-23 14:27:59+00:00
type=TRANSFER Mbytes=12219.0 Mbytes/s=1585.8
task_id= ( 9f5467a2-e192-11e6-9d3a-22000a1e3b52 ) arnoldg#macos -> ncsa#BlueWaters
request_time 2017-01-23 17:37:34+00:00
completed    2017-01-23 17:37:38+00:00
type=TRANSFER Mbytes=87478.0 Mbytes/s=19728.2
task_id= ( de2554aa-e183-11e6-9d39-22000a1e3b52 ) ncsa#jyc -> ncsa#BlueWaters
request_time 2017-01-23 15:51:57+00:00
completed    2017-01-23 15:52:01+00:00
type=TRANSFER Mbytes=1.0 Mbytes/s=0.2
task_id= ( d60dd918-e183-11e6-9d39-22000a1e3b52 ) ncsa#jyc -> ncsa#BlueWaters
request_time 2017-01-23 15:51:43+00:00
completed    2017-01-23 15:51:47+00:00
type=TRANSFER Mbytes=2.4 Mbytes/s=0.7
tas